In [1]:
from pendulum_forces import MagneticPendulumInvSpherical, MagneticPendulumXY
import numpy as np
import sympy as sp
from scipy.constants import g as g_value

In [2]:
P = MagneticPendulumInvSpherical(4)
# P = MagneticPendulumXY(4)

const1_subs = {
    'm': 1,
    'g': g_value,
    'L': g_value,
    'R': 2,
    'h': 0.5,
    'b': 0.1,   
}
magnet_strength = 10

magnet_polarity_vals = dict((p.name, magnet_strength) for p in P.polarity_list)
constant_subs = const1_subs | magnet_polarity_vals

bob = P.create_bob()

In [26]:
t_max = 100
dt = 0.001

## specifically for spherical
to_x = lambda r,θ,ϕ: r*np.sin(θ)*np.cos(ϕ)
to_y = lambda r,θ,ϕ: r*np.sin(θ)*np.sin(ϕ)
to_z = lambda r,θ,ϕ: -r*np.cos(θ)
# to_cartesian = lambda r,θ,ϕ: [to_x(r,θ,ϕ), to_y(r,θ,ϕ), to_z(r,θ,ϕ)]


starting_pos_1 = np.array([
    constant_subs['L'],
    0.19166740685923828*3/4 * 5,
    np.pi/10
])
####

starting_pos_array = np.array([
    starting_pos_1
])

final_pos_array = np.zeros(starting_pos_array.shape)


In [17]:
starting_pos_array[0]

array([9.80665   , 0.71875278, 0.31415927])

In [20]:
starting_pos_1 = np.array([
    constant_subs['L'],
    0.19166740685923828*3/4 * 5,
    np.pi/10
])




[9.80665    0.71875278 0.31415927]


array([[9.80665000e+00, 1.88929114e-01, 2.26376727e-03]])

In [37]:
resolution=30
r_const = constant_subs['L']
phi = np.linspace(0, 2*np.pi, 2*resolution)
theta = np.linspace(0, np.pi, resolution)

theta_mesh, phi_mesh = np.meshgrid(theta, phi)



In [57]:
phi_mesh.shape #??????????????

(60, 30)

In [54]:
np.stack([theta_mesh, phi_mesh])

array([[[0.        , 0.10833078, 0.21666156, ..., 2.92493109,
         3.03326187, 3.14159265],
        [0.        , 0.10833078, 0.21666156, ..., 2.92493109,
         3.03326187, 3.14159265],
        [0.        , 0.10833078, 0.21666156, ..., 2.92493109,
         3.03326187, 3.14159265],
        ...,
        [0.        , 0.10833078, 0.21666156, ..., 2.92493109,
         3.03326187, 3.14159265],
        [0.        , 0.10833078, 0.21666156, ..., 2.92493109,
         3.03326187, 3.14159265],
        [0.        , 0.10833078, 0.21666156, ..., 2.92493109,
         3.03326187, 3.14159265]],

       [[0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.10649467, 0.10649467, 0.10649467, ..., 0.10649467,
         0.10649467, 0.10649467],
        [0.21298933, 0.21298933, 0.21298933, ..., 0.21298933,
         0.21298933, 0.21298933],
        ...,
        [6.07019597, 6.07019597, 6.07019597, ..., 6.07019597,
         6.07019597, 6.07019597],
        [6.1

In [42]:
def get_final_position(θ_start, ϕ_start):
    starting_conditions = [r_const, θ_start, ϕ_start, 0,0,0]
    final_pos_spherical = P.get_path(starting_conditions, t_max=t_max, dt=0.01,
                      bob=bob, **constant_subs)[:,-1]
    return final_pos_spherical[1], final_pos_spherical[2]

In [49]:
np.array([r_const, θ_start, ϕ_start, 0,0,0])

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (6,) + inhomogeneous part.

In [55]:
for thing in np.stack([theta_mesh, phi_mesh]):
    print(thing)

[[0.         0.10833078 0.21666156 ... 2.92493109 3.03326187 3.14159265]
 [0.         0.10833078 0.21666156 ... 2.92493109 3.03326187 3.14159265]
 [0.         0.10833078 0.21666156 ... 2.92493109 3.03326187 3.14159265]
 ...
 [0.         0.10833078 0.21666156 ... 2.92493109 3.03326187 3.14159265]
 [0.         0.10833078 0.21666156 ... 2.92493109 3.03326187 3.14159265]
 [0.         0.10833078 0.21666156 ... 2.92493109 3.03326187 3.14159265]]
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.10649467 0.10649467 0.10649467 ... 0.10649467 0.10649467 0.10649467]
 [0.21298933 0.21298933 0.21298933 ... 0.21298933 0.21298933 0.21298933]
 ...
 [6.07019597 6.07019597 6.07019597 ... 6.07019597 6.07019597 6.07019597]
 [6.17669064 6.17669064 6.17669064 ... 6.17669064 6.17669064 6.17669064]
 [6.28318531 6.28318531 6.28318531 ... 6.28318531 6.28318531 6.28318531]]


In [46]:
θ_start, ϕ_start = theta_mesh[0], phi_mesh[0]

P.get_path(np.array([r_const, θ_start, ϕ_start, 0,0,0]), t_max=t_max, dt=0.01,
                      bob=bob, **constant_subs)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (6,) + inhomogeneous part.

In [43]:
np.array([get_final_position(*start) for start in zip(theta_mesh, phi_mesh)])

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (6,) + inhomogeneous part.

In [41]:
[get_final_position(*start) for start in zip(theta_mesh, phi_mesh)]

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (6,) + inhomogeneous part.